In [ ]:
!pip install ultralytics --quiet

Conversão das labels

In [ ]:
import os
from PIL import Image

# Caminho para as pastas de imagens e labels
images_path = "/content/drive/MyDrive/Hackaton/Dataset/val/images"
labels_path = "/content/drive/MyDrive/Hackaton/Dataset/val/labels"

# ID da classe (0 para "faca")
class_id = 0

# Função para converter as labels
def convert_labels(image_width, image_height, x_min, y_min, x_max, y_max):
    # Calcular as coordenadas normalizadas
    x_center = ((x_min + x_max) / 2) / image_width
    y_center = ((y_min + y_max) / 2) / image_height
    width = (x_max - x_min) / image_width
    height = (y_max - y_min) / image_height
    return f"{class_id} {x_center} {y_center} {width} {height}"

# Percorrer todos os arquivos de labels
for label_file in os.listdir(labels_path):
    if label_file.endswith(".txt"):
        label_file_path = os.path.join(labels_path, label_file)

        # Ler o conteúdo do arquivo
        with open(label_file_path, "r") as file:
            lines = file.readlines()

        # Obter o nome da imagem correspondente
        image_file = label_file.replace(".txt", ".jpg")
        image_path = os.path.join(images_path, image_file)

        # Verificar se a imagem existe
        if not os.path.exists(image_path):
            print(f"Imagem faltando para o label: {image_file}")
            continue

        # Obter as dimensões da imagem
        with Image.open(image_path) as img:
            image_width, image_height = img.size

        # Escrever as labels no formato YOLO
        with open(label_file_path, "w") as file:
            for line in lines:
                # Extrair os valores atuais
                parts = line.strip().split()
                class_name = parts[0]  # "Knife"
                x_min = float(parts[1])  # x_min
                y_min = float(parts[2])  # y_min
                x_max = float(parts[3])  # x_max
                y_max = float(parts[4])  # y_max

                # Converter para o formato YOLO
                yolo_label = convert_labels(image_width, image_height, x_min, y_min, x_max, y_max)

                # Escrever a linha no formato YOLO
                file.write(yolo_label + "\n")

print("Conversão das labels concluída!")

Conversão das labels concluída!


In [ ]:
from ultralytics import YOLO
import os

# 1. Configuração do Dataset
# Certifique-se de que o dataset está organizado da seguinte forma:
# /dataset/
#    ├── train/
#    │   ├── images/
#    │   └── labels/
#    └── val/
#        ├── images/
#        └── labels/

# Caminho para o dataset
dataset_path = "/content/drive/MyDrive/Hackaton/Dataset"

# 2. Criar o arquivo data.yaml
data_yaml = f"""

train: {dataset_path}/train/images
val: {dataset_path}/val/images

# Número de classes
nc: 1

# Nomes das classes
names: ['Knife']
"""

# Salvar o arquivo data.yaml
with open("/content/drive/MyDrive/Hackaton/data.yaml", "w") as file:
    file.write(data_yaml)

# 3. Verificar a estrutura do dataset
def verificar_dataset(dataset_path):
    print("Verificando estrutura do dataset...")
    pastas_necessarias = ["/content/drive/MyDrive/Hackaton/Dataset/train/images", "/content/drive/MyDrive/Hackaton/Dataset/train/labels", "/content/drive/MyDrive/Hackaton/Dataset/val/images", "/content/drive/MyDrive/Hackaton/Dataset/val/labels"]
    for pasta in pastas_necessarias:
        if not os.path.exists(os.path.join(dataset_path, pasta)):
            raise Exception(f"Pasta faltando: {pasta}")
    print("Estrutura do dataset está correta!")

verificar_dataset(dataset_path)

# 4. Carregar um modelo pré-treinado
# YOLOv8n é a versão "nano" (leve e rápida)
model = YOLO("yolo11m.pt")

# 5. Treinar o modelo
print("Iniciando o treinamento...")
results = model.train(
    data="/content/drive/MyDrive/Hackaton/data.yaml",  # Caminho para o arquivo de configuração
    epochs=50,         # Número de épocas
    imgsz=640,         # Tamanho da imagem (640x640 é comum)
    batch=8,           # Tamanho do batch (ajuste conforme sua GPU)
    name="yolov11_knife_detection",  # Nome do experimento
    patience=25,       # Parar o treinamento se não houver melhoria em 10 épocas
    device="cpu",        # Usar GPU (se disponível). Use "cpu" para treinar na CPU.
)

# 6. Avaliar o modelo
print("Avaliando o modelo...")
metrics = model.val()  # Avaliar no conjunto de validação
print(f"mAP50-95: {metrics.box.map}")  # Exibir a métrica mAP

# 7. Testar o modelo em uma imagem de exemplo
print("Testando o modelo em uma imagem de exemplo...")
test_image_path = os.path.join(dataset_path, "/content/drive/MyDrive/Hackaton/Dataset/val/images/096c39dfb17068cf.jpg")  # Substitua pelo caminho de uma imagem de teste
results = model.predict(test_image_path, save=True, conf=0.5)  # conf = limite de confiança

# Exibir os resultados
for result in results:
    result.show()  # Exibir a imagem com as detecções
    result.save()  # Salvar a imagem com as detecções

# 8. Exportar o modelo para ONNX (opcional)
print("Exportando o modelo para ONNX...")
model.export(format="onnx")

# 9. Mensagem final
print("Treinamento concluído!")
print(f"Resultados salvos em: {os.path.abspath('runs/detect/yolov8_knife_detection')}")

Script para Testar o Modelo em Várias Imagens

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from ultralytics import YOLO

# Carregar o modelo treinado
model = YOLO("/content/drive/MyDrive/Hackaton/yolov11n/yolo11m.pt")

# Caminho para as imagens de validação
val_images_path = "/content/drive/MyDrive/Hackaton/Dataset/train/images"

# Testar o modelo em todas as imagens de validação
for image_name in os.listdir(val_images_path):
    if image_name.endswith(".jpg") or image_name.endswith(".png"):
        image_path = os.path.join(val_images_path, image_name)
        print(f"Testando o modelo na imagem: {image_name}")
        results = model.predict(image_path, save=True, conf=0.5)  # conf = limite de confiança
        for result in results:
            result.show()  # Exibir a imagem com as detecções

print("Teste concluído!")

Testar o modelo em um video

In [ ]:
from ultralytics import YOLO

# 1. Carregar o modelo treinado
model = YOLO("/content/drive/MyDrive/Hackaton/yolov11n/yolo11m.pt")

# 2. Especificar o caminho do vídeo
video_path = "/content/drive/MyDrive/Hackaton/video.mp4"  # Substitua pelo caminho do seu vídeo

# target_classes = [43,76]  # Substitua 47 pelo ID correto da classe faca no seu modelo

# 3. Processar o vídeo
results = model.predict(
    source=video_path,  # Caminho do vídeo
    conf=0.5,           # Limite de confiança para as detecções
    save=True,          # Salvar o vídeo com as detecções
    show=True,           # Exibir o vídeo em tempo real (opcional)
    #classes=target_classes  # Filtrar as detecções por classes específicas
)

# 4. Mensagem final
print("Processamento do vídeo concluído!")
print(f"Vídeo com detecções salvo em: {results[0].save_dir}")